In [12]:
from pathlib import Path

def read_and_sort_concerts(file_path):
    concerts = {}
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith('#') and line.strip():
                genre, count = line.split(':')
                concerts[genre.strip()] = int(count.strip())
    
    sorted_concerts = sorted(concerts.items(), key=lambda item: item[1], reverse=True)
    return sorted_concerts

file_path = Path.cwd().parent /'Data'/'grupee_data'/'n_concerts.txt'
sorted_concerts = read_and_sort_concerts(file_path)
for genre, count in sorted_concerts:
    print(f"{genre}: {count}")

Spirituality & Religion: 60
International Pop: 40
Indie Rock/Rock pop: 38
Dancefloor: 36
Singer & Songwriter: 30
Indie Pop: 30
Kids & Family: 30
Pop: 29
Techno/House: 29
Rap/Hip Hop: 28
Contemporary R&B: 27
Indie Rock: 25
Classical: 24
Electro Pop/Electro Rock: 24
R&B: 24
Folk: 23
Vocal jazz: 23
Romantic: 22
Dubstep: 22
Dub: 22
Electro: 20
Indie Pop/Folk: 18
Alternative: 18
Hard Rock: 18
Metal: 15
Rock: 14
Blues: 14
Electro Hip Hop: 13
Country Blues: 13
East Coast: 13
Comedy: 12
West Coast: 12
Jazz: 12
Trance: 11
Musicals: 10
Dancehall/Ragga: 10
Latin Music: 10
Old school soul: 9
Contemporary Soul: 9
Classic Blues: 9
Jazz Hip Hop: 8
Instrumental jazz: 8
Dance: 8
Soul & Funk: 8
Alternative Country: 6
Rock & Roll/Rockabilly: 6
Reggae: 5
Asian Music: 4
African Music: 4
Opera: 4
Chill Out/Trip-Hop/Lounge: 3
Kids: 3
Bluegrass: 2
Acoustic Blues: 2
Traditional Country: 2
Country: 2
Baroque: 1
Bolero: 1
Indian Music: 1
Electric Blues: 1
Soundtracks: 0
Disco: 0
Old School: 0
Urban Cowboy: 0
Tro

In [ ]:
from utility import graph
import json
import csv
import numpy as np
from alive_progress import alive_bar

current_path = Path.cwd().parent 
concerts = graph.read_concerts(current_path /'Data'/'grupee_data'/'n_concerts.txt')
# Perform the division and multiplication while maintaining the tuple structure
concert_per_two_weeks_scaler = [((vc[1] / 52.1429) * 2.0) for vc in concerts]
# visualizations.visualize_concerts(concert_per_two_weeks, "two weeks")
# print("visit per two weeks", concert_per_two_weeks)

# Extract the preferences of the grupees
preferences_path = current_path /'Data'/'grupee_data'/'preferences.json'
with open(preferences_path, 'r') as file:
    preferences = json.load(file)

# Read the connections between grupees
friend_pairs = []
friends_path = current_path /'Data'/'grupee_data'/'friends.csv'
with open(friends_path, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        friend_pairs.append(row)
    friend_pairs.pop(0) # first line contains an unnecessary comment

# probability to infect a friend depending on preferences
both_like = 393/1000
one_like = 18/1000
neither_like = 2/1000
genre_count = len(preferences[friend_pairs[0][0]]) 
risk_per_id = [[0] * genre_count for _ in range(len(preferences.keys()))]
with alive_bar(len(friend_pairs), title='Processing Friend IDs') as bar:
    for f_id in friend_pairs: # f_id is a list of two friend ids
        risk_per_id_row = [0] * genre_count
        pref_1 = preferences[f_id[0]]
        pref_2 = preferences[f_id[1]]
        for i, (p1, p2) in enumerate(zip(pref_1, pref_2)): # iterate over the prefereed genres of the two friends
            if p1 == '1' and p2 == '1':
                risk_per_id_row[i] += both_like
            elif p1 == '1' or p2 == '1':
                risk_per_id_row[i] += one_like
            else:
                risk_per_id_row[i] += neither_like
        # Element wise addition of the risk per genre
        risk_per_id[int(f_id[0])] = [x + y for x, y in zip(risk_per_id[int(f_id[0])], risk_per_id_row)]
        risk_per_id[int(f_id[1])] = [x + y for x, y in zip(risk_per_id[int(f_id[1])], risk_per_id_row)]
        bar()
# scale the risk per genre of each grupee by the number of concerts per two weeks
scaled_risk_per_id = [[x * y for x, y in zip(row, concert_per_two_weeks_scaler)] for row in risk_per_id] # theoretically could scale by concerts instead
# Sum the elements of each row in scaled_risk_per_ids_concert
sum_scaled_risk_per_id = [sum(row) for row in scaled_risk_per_id]
# Calculate the number of top elements to select (12% of the total length)
top_percentage = 0.12
num_top_elements = int(len(sum_scaled_risk_per_id) * top_percentage)

# Get the indices of the top elements
top_indices = np.argsort(sum_scaled_risk_per_id)[-num_top_elements:][::-1]

# Get the values of the top elements
top_values = [sum_scaled_risk_per_id[i] for i in top_indices]

print("Top 12% indices:", top_indices)
print("Top 12% values:", top_values)
# top 12% of the riskiest people to be affected by the virus (due to visitation of concerts with friends)
# FROM HERE ON: 
# top_indices: list of ids of the top 12% of the riskiest people to be affected by the virus (sorted by risk)
# top_values: list of the corresponding risks of the top 12% of the riskiest people to be affected by the virus

FileNotFoundError: [Errno 2] No such file or directory: 'problem-3\\Data\\grupee_data\\preferences.json'